# Model regresji liniowej wykorzystujący trendy, sezonowość, promocje oraz lagi

### Zaimportowanie bilbiotek i deklaracja używanych później funkcji

In [1]:
from warnings import simplefilter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.signal import periodogram
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from pathlib import Path
simplefilter("ignore")

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(20, 7))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)
%config InlineBackend.figure_format = 'retina'


def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax


def make_lags(ts, lags):
    return pd.concat(
        {f'y_lag_{i}': ts.shift(i) for i in range(1, lags + 1)},axis=1)

def make_multistep_target(ts, steps):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)

### Wczytanie danych

In [2]:
#load the data (kaggle)
# comp_dir = Path('../input/store-sales-time-series-forecasting')
# store_sales = pd.read_csv(
#     comp_dir / 'train.csv',
#     usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
#     dtype={
#         'store_nbr': 'category',
#         'family': 'category',
#         'sales': 'float32',
#         'onpromotion': 'uint32',
#     },
#     parse_dates=['date'],
#     infer_datetime_format=True,
# )
# df_test = pd.read_csv(
#     comp_dir / 'test.csv',
#     dtype={
#         'store_nbr': 'category',
#         'family': 'category',
#         'onpromotion': 'uint32',
#     },
#     parse_dates=['date'],
#     infer_datetime_format=True,
# )

#load the data (local)

store_sales = pd.read_csv('train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)

df_test = pd.read_csv('test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)

df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()
store_sales

sales  onpromotion
store_nbr family     date                              
1         AUTOMOTIVE 2013-01-01   0.000000            0
                     2013-01-02   2.000000            0
                     2013-01-03   3.000000            0
                     2013-01-04   3.000000            0
                     2013-01-05   5.000000            0
...                                    ...          ...
9         SEAFOOD    2017-08-11  23.830999            0
                     2017-08-12  16.859001            4
                     2017-08-13  20.000000            0
                     2017-08-14  17.000000            0
                     2017-08-15  16.000000            0

[3000888 rows x 2 columns]

### Przygotowanie trendów i sezonowości sprzedaży jako cech do wytrenowania modelu

In [3]:
y = store_sales.unstack(['store_nbr', 'family'])
fourier = CalendarFourier("m",4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    drop=True,
    additional_terms=[fourier]
)
X = dp.in_sample()
X

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
date,,,,,,,,,,,,,,,,
2013-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2013-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2013-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649
2013-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758
2013-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,1680.0,0.0,0.0,0.0,0.0,0.0,1.0,0.897805,-0.440394,-0.790776,-0.612106,-0.201299,0.979530,0.968077,-0.250653
2017-08-12,1.0,1681.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790776,-0.612106,-0.968077,-0.250653,0.394356,0.918958,0.485302,-0.874347
2017-08-13,1.0,1682.0,1.0,0.0,0.0,0.0,0.0,0.0,0.651372,-0.758758,-0.988468,0.151428,0.848644,0.528964,-0.299363,-0.954139


### Przygotowanie lagów z oryginalnych danych o sprzedaży i promocjach

In [4]:
y = store_sales.unstack(['store_nbr', 'family']).loc['2017', 'sales']
all_promotion = store_sales.unstack(['store_nbr', 'family']).loc(axis=1)['onpromotion'].loc['2017']
X_lags = make_lags(y, lags=1)

X_promo = pd.concat([
    make_lags(all_promotion, lags=1),
    all_promotion,
], axis=1)


### Połączenie wszystkich cech w jedną macierz

In [5]:
X_whole = pd.concat([X, X_lags, X_promo], axis=1).dropna()


### Transformacja danych treningowych (1 kolumna to jedna kategoria)

In [6]:
y_whole = store_sales.unstack(['store_nbr', 'family']).loc['2017', 'sales']

### Stworzenie multistep target

In [7]:
y = make_multistep_target(y, steps=16).dropna()

### Połączenie tabel

In [8]:
y, X = y.align(X_whole, join='inner', axis = 0)

y

y_step_1                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-02        5.0       0.0    0.0    1434.0   0.0   166.819000   
2017-01-03        4.0       0.0    4.0    3081.0   2.0   519.348022   
2017-01-04        1.0       0.0    4.0    3039.0   2.0   543.250977   
2017-01-05        2.0       0.0    3.0    2617.0   0.0   533.479980   
2017-01-06        2.0       0.0   10.0    2761.0   0.0   442.910004   
...               ...       ...    ...       ...   ...          ...   
2017-07-27        5.0       0.0    1.0    2002.0   0.0   321.000000   
2017-07-28        7.0       0.0    2.0    2358.0   0.0   403.644989   
2017-07-29        4.0       0.0    3.0    2161.0   0.0   330.035004   
2017-07-30        1.0       0.0    2.0    1212.0   0.0   153.807999   
2017-07-31        8.0       0.0    3.0    2414.0   1.0   370.993988   

                                                     ... y_step_16  \
store_nbr                                            ...         9   
family     CELEBRATION CLEANING   DAIRY        DELI  ... MAGAZINES   
date                                                 ...             
2017-01-02         0.0    332.0   376.0   44.980000  ...       2.0   
2017-01-03        15.0    952.0  1045.0  209.300003  ...       8.0   
2017-01-04        17.0   1055.0  1029.0  135.944000  ...       3.0   
2017-01-05        40.0    918.0   853.0  137.005997  ...       1.0   
2017-01-06         9.0    799.0   927.0  162.621994  ...       5.0   
...                ...      ...     ...         ...  ...       ...   
2017-07-27        21.0    660.0   722.0  116.137001  ...       5.0   
2017-07-28        20.0    714.0   711.0  178.408997  ...       2.0   
2017-07-29         6.0    667.0   676.0  122.680000  ...       3.0   
2017-07-30         0.0    238.0   316.0   54.296001  ...      12.0   
2017-07-31         3.0    678.0   727.0  189.110001  ...      11.0   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-02  415.770996         406.0          3.0                     9.0   
2017-01-03  351.690002         357.0          5.0                     9.0   
2017-01-04  551.434021         309.0          3.0                     7.0   
2017-01-05  285.218994         275.0          6.0                    13.0   
2017-01-06  479.380005         580.0         11.0                     9.0   
...                ...           ...          ...                     ...   
2017-07-27  309.244995         373.0         11.0                     2.0   
2017-07-28  260.298004         400.0          7.0                    10.0   
2017-07-29  327.205994         510.0          2.0                     9.0   
2017-07-30  330.975006         445.0          2.0                    14.0   
2017-07-31  449.227997         522.0          6.0                     6.0   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-02  349.972015     111.331001  2441.391113                        0.0   
2017-01-03  354.292999     147.319000  1298.261963                        2.0   
2017-01-04  297.868011     115.235001  1297.201050                        0.0   
2017-01-05  408.058990     136.686996  1239.003052                        1.0   
2017-01-06  667.434021     213.098999  2178.283936                        2.0   
...        

In [9]:
X

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)",...,"(9, MAGAZINES)","(9, MEATS)","(9, PERSONAL CARE)","(9, PET SUPPLIES)","(9, PLAYERS AND ELECTRONICS)","(9, POULTRY)","(9, PREPARED FOODS)","(9, PRODUCE)","(9, SCHOOL AND OFFICE SUPPLIES)","(9, SEAFOOD)"
date,,,,,,,,,,,,,,,,,,,,,
2017-01-02,1.0,1459.0,0.0,1.0,0.0,0.0,0.0,0.0,0.201299,0.979530,...,0.0,0.0,13.0,0.0,0.0,2.0,1.0,4.0,0.0,0.0
2017-01-03,1.0,1460.0,0.0,0.0,1.0,0.0,0.0,0.0,0.394356,0.918958,...,0.0,0.0,11.0,0.0,0.0,1.0,2.0,150.0,0.0,0.0
2017-01-04,1.0,1461.0,0.0,0.0,0.0,1.0,0.0,0.0,0.571268,0.820763,...,0.0,0.0,15.0,0.0,0.0,1.0,8.0,9.0,0.0,0.0
2017-01-05,1.0,1462.0,0.0,0.0,0.0,0.0,1.0,0.0,0.724793,0.688967,...,0.0,21.0,8.0,0.0,0.0,1.0,1.0,5.0,0.0,1.0
2017-01-06,1.0,1463.0,0.0,0.0,0.0,0.0,0.0,1.0,0.848644,0.528964,...,0.0,0.0,12.0,0.0,0.0,22.0,2.0,7.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-27,1.0,1665.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.848644,0.528964,...,0.0,21.0,9.0,0.0,0.0,0.0,1.0,9.0,0.0,0.0
2017-07-28,1.0,1666.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.724793,0.688967,...,0.0,0.0,9.0,0.0,0.0,22.0,3.0,8.0,4.0,0.0
2017-07-29,1.0,1667.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.571268,0.820763,...,0.0,0.0,9.0,0.0,0.0,0.0,1.0,8.0,7.0,4.0


### Ekstrakcja cech, które będą użyte w multistep targetingu

In [10]:
X_fore = X_whole.loc['2017-08']
X_fore

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)",...,"(9, MAGAZINES)","(9, MEATS)","(9, PERSONAL CARE)","(9, PET SUPPLIES)","(9, PLAYERS AND ELECTRONICS)","(9, POULTRY)","(9, PREPARED FOODS)","(9, PRODUCE)","(9, SCHOOL AND OFFICE SUPPLIES)","(9, SEAFOOD)"
date,,,,,,,,,,,,,,,,,,,,,
2017-08-01,1.0,1670.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,1.000000,...,0.0,0.0,9.0,0.0,1.0,0.0,0.0,151.0,9.0,0.0
2017-08-02,1.0,1671.0,0.0,0.0,0.0,1.0,0.0,0.0,0.201299,0.979530,...,0.0,0.0,9.0,0.0,0.0,0.0,1.0,8.0,7.0,0.0
2017-08-03,1.0,1672.0,0.0,0.0,0.0,0.0,1.0,0.0,0.394356,0.918958,...,0.0,21.0,10.0,0.0,0.0,1.0,0.0,8.0,6.0,0.0
2017-08-04,1.0,1673.0,0.0,0.0,0.0,0.0,0.0,1.0,0.571268,0.820763,...,0.0,0.0,11.0,0.0,0.0,23.0,3.0,8.0,7.0,0.0
2017-08-05,1.0,1674.0,0.0,0.0,0.0,0.0,0.0,0.0,0.724793,0.688967,...,0.0,0.0,12.0,0.0,0.0,1.0,1.0,6.0,9.0,3.0
2017-08-06,1.0,1675.0,1.0,0.0,0.0,0.0,0.0,0.0,0.848644,0.528964,...,0.0,0.0,12.0,0.0,0.0,0.0,1.0,7.0,9.0,0.0
2017-08-07,1.0,1676.0,0.0,1.0,0.0,0.0,0.0,0.0,0.937752,0.347305,...,0.0,0.0,12.0,0.0,0.0,0.0,1.0,6.0,10.0,0.0
2017-08-08,1.0,1677.0,0.0,0.0,1.0,0.0,0.0,0.0,0.988468,0.151428,...,0.0,0.0,11.0,0.0,0.0,0.0,1.0,144.0,7.0,0.0
2017-08-09,1.0,1678.0,0.0,0.0,0.0,1.0,0.0,0.0,0.998717,-0.050649,...,0.0,0.0,10.0,0.0,0.0,1.0,1.0,6.0,8.0,0.0


### Wytrenowanie modelu oraz użycie go z przygotowanymi danymi

In [11]:
linear_model = LinearRegression(fit_intercept=False).fit(X, y)

linear_y_forecast = pd.DataFrame(linear_model.predict(X_fore), index=X_fore.index, columns = y.columns).clip(0.0)

### Uporządkowanie danych

In [12]:
linear_y_forecast.loc['2017-08-15']

           store_nbr  family                    
y_step_1   1          AUTOMOTIVE                       8.266289
                      BABY CARE                        0.000000
                      BEAUTY                           2.979640
                      BEVERAGES                     2168.069883
                      BOOKS                            2.304960
                                                       ...     
y_step_16  9          POULTRY                        354.993923
                      PREPARED FOODS                  79.254750
                      PRODUCE                       1347.178988
                      SCHOOL AND OFFICE SUPPLIES      52.222656
                      SEAFOOD                          0.000000
Name: 2017-08-15, Length: 28512, dtype: float64

### Formatowanie wyników

In [13]:
linear_forecast = linear_y_forecast.loc['2017-08-15']
linear_forecast.index.names=['date', 'store_nbr', 'family']
forecast_formatted = linear_forecast.unstack('date', 'family').stack('date')
forecast_formatted.index = df_test.index

pd.DataFrame(forecast_formatted, columns = ['sales'])

sales
store_nbr family     date                 
1         AUTOMOTIVE 2017-08-16   8.266289
                     2017-08-17   7.848674
                     2017-08-18   1.643776
                     2017-08-19   6.778326
                     2017-08-20  10.137909
...                                    ...
9         SEAFOOD    2017-08-27  23.363690
                     2017-08-28  22.215546
                     2017-08-29  29.652282
                     2017-08-30  12.024286
                     2017-08-31   0.000000

[28512 rows x 1 columns]

### Zapisanie predykcji w submission.csv

In [14]:
forecast_formatted = pd.DataFrame(forecast_formatted, columns = ['sales'])
y_submit = forecast_formatted.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)